In [1]:
%matplotlib inline

In [2]:
import os 
import sys
import json
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import neurokit as nk
import scipy as sc
import math 
import scipy.signal as ss
import warnings
import itertools

from scipy import signal
warnings.filterwarnings('ignore')

In [3]:
#Loading the curated dataframe of signals
curated_signals = joblib.load("/home/camar_temp/git-repos/buckets/bkt_inv_braintherapy_files/brain_therapy/BT_MVP_Content_Pilot/MVP_Focused_Attention/pkls/data_pkls/FA_corrected_signals_amplitude.pkl")

In [4]:
#Converting the dictionary of signals into a single dataframe
p = curated_signals.keys()
curated_df = pd.DataFrame([])
for part in p:
    data = curated_signals[part]
    data['Participant'] = part.split("-")[3]
    curated_df = pd.concat([curated_df,data])
curated_df.set_index(["Participant","Datetime"], inplace = True)
curated_df.head(3)

RSP       PPG     1-SKTA       ECG  \
Participant Datetime                                                           
0001        2019-07-10 15:22:31.000 -1.263123  0.003052  12.941162 -0.234833   
            2019-07-10 15:22:31.004 -1.261902  0.003662  12.945435 -0.226593   
            2019-07-10 15:22:31.008 -1.264648  0.002136  12.946045 -0.215607   

                                          EDA  
Participant Datetime                           
0001        2019-07-10 15:22:31.000  0.041124  
            2019-07-10 15:22:31.004  0.042649  
            2019-07-10 15:22:31.008  0.044175

In [5]:
# Loading the dataframe of the events of each participant
log_events = joblib.load("/home/camar_temp/git-repos/buckets/bkt_inv_braintherapy_files/brain_therapy/BT_MVP_Content_Pilot/MVP_Focused_Attention/pkls/data_pkls/log_events.pkl")
log_events = log_events.reset_index().rename(columns = {'participant':"Participant", "time":"Datetime"}).set_index(["Participant","Datetime"])
log_events.head(3)

type           context
Participant Datetime                                  
0001        2019-07-10 15:25:32   2D  StartFrontBuffer
            2019-07-10 15:27:32   2D   StopFrontBuffer
            2019-07-10 15:27:32   2D           StartLL

In [6]:
#Merging the log_events and signals dataframe
merged_df = curated_df.merge(log_events, how = "left", right_index = True, left_index = True)
#Filling nans
merged_df['context'] = merged_df['context'].ffill()
merged_df['type'] = merged_df['type'].ffill()
merged_df['context'] = merged_df['context'].fillna("Garbage")
merged_df['type'] = merged_df['type'].fillna("Garbage")
merged_df = merged_df.reset_index().set_index(["Participant","type","context","Datetime"])
merged_df.head(2)

RSP       PPG  \
Participant type    context Datetime                                      
0001        Garbage Garbage 2019-07-10 15:22:31.000 -1.263123  0.003052   
                            2019-07-10 15:22:31.004 -1.261902  0.003662   

                                                        1-SKTA       ECG  \
Participant type    context Datetime                                       
0001        Garbage Garbage 2019-07-10 15:22:31.000  12.941162 -0.234833   
                            2019-07-10 15:22:31.004  12.945435 -0.226593   

                                                          EDA  
Participant type    context Datetime                           
0001        Garbage Garbage 2019-07-10 15:22:31.000  0.041124  
                            2019-07-10 15:22:31.004  0.042649

#### Removing "Start" and "Stop" for easier groupings

In [7]:
def change_events(df):
    df = df.reset_index("context")
    s = [str(x).replace('Start','') for x in df['context'].values]
    s = [str(x).replace('Stop','') for x in s]
    s = [str(x).replace('nan','Garbage') for x in s]
    df['context'] = s
#     df = df[df['context']!="FrontBuffer"]
#     df = df[df['context']!="EndBuffer"]
#     df = df[df['context']!="Garbage"]
    df = df.reset_index().set_index(["Participant","type","context","Datetime"])
    return df

In [8]:
merged_df = change_events(merged_df)

In [9]:
merged_df.head(2)

RSP       PPG  \
Participant type    context Datetime                                      
0001        Garbage Garbage 2019-07-10 15:22:31.000 -1.263123  0.003052   
                            2019-07-10 15:22:31.004 -1.261902  0.003662   

                                                        1-SKTA       ECG  \
Participant type    context Datetime                                       
0001        Garbage Garbage 2019-07-10 15:22:31.000  12.941162 -0.234833   
                            2019-07-10 15:22:31.004  12.945435 -0.226593   

                                                          EDA  
Participant type    context Datetime                           
0001        Garbage Garbage 2019-07-10 15:22:31.000  0.041124  
                            2019-07-10 15:22:31.004  0.042649

## ECG

In [10]:
participants = merged_df.index.get_level_values("Participant").unique()
good_peaks = joblib.load("/home/camar_temp/git-repos/buckets/bkt_inv_braintherapy_files/brain_therapy/BT_MVP_Content_Pilot/MVP_Focused_Attention/pkls/data_pkls/FA_r_peaks_index.pkl")

##### Computing ECG Heart Rate and Heart Rate Variability

Computing the time differences of every peak in miliseconds

In [11]:
ecg_hr_features = pd.DataFrame([])
ecg_hrv_features = pd.DataFrame([])
time_diff_df = pd.DataFrame([])
for participant in participants:
    data = merged_df.loc(axis = 0)[participant,:,:,:]
    ecg_peaks = good_peaks["BT-MVP-FA-{}".format(participant)]
    time_stamps = data.index.get_level_values('Datetime')[ecg_peaks]
    d = data.loc(axis = 0)[participant,:,:,time_stamps]
    
    #Computing time differences
    try:
        d['time_differences'] = np.concatenate([np.diff(time_stamps)/np.timedelta64(1,'ms'), [0]])
    except:
        d['time_differences'] = np.concatenate([np.diff(time_stamps)/np.timedelta64(1,'ms'), [0,0]])

    df = d[['time_differences']]
    
    #Taking out time differences that is more than the upper bound and less than the lower bound
    diff_mean = np.mean(df.time_differences)
    upper_threshold = 1200          # diff_mean + (4*np.std(df.time_differences))
    lower_threshold = 400          # diff_mean - (4*np.std(df.time_differences))
    df = df[(df.time_differences<upper_threshold) & (df.time_differences>lower_threshold)]
    
    # Drop NaN contexts
    temp = df.reset_index("context")
    temp = temp[pd.notnull(temp['context'])]
    df = temp.reset_index().set_index(["Participant","type","context"])
    time_diff_df = pd.concat([time_diff_df, df])
    

In [12]:
time_diff_df.head(2)

Datetime  time_differences
Participant type    context                                          
0001        Garbage Garbage 2019-07-10 15:22:31.700            1064.0
                    Garbage 2019-07-10 15:22:32.764            1024.0

### ECG_HR features

In [13]:
ecg_hr_features = time_diff_df.groupby(["Participant","type","context"], 
                                        sort = True).mean()
ecg_hr_features['ecg_heart_rate'] = (60/ecg_hr_features['time_differences'])*1000
ecg_hr_features.drop("time_differences",axis = 1, inplace = True)
ecg_hr_features.head(2)

ecg_heart_rate
Participant type context                    
0001        2D   EndBuffer         67.254897
                 FrontBuffer       64.474210

### ECG_HRV features

In [14]:
ecg_hrv_features = time_diff_df.groupby(["Participant","type","context"], sort = True).std()
ecg_hrv_features.rename(columns = {"time_differences":"ecg_hrv_std"}, inplace = True)
ecg_hrv_features.head(2)

ecg_hrv_std
Participant type context                 
0001        2D   EndBuffer      95.179201
                 FrontBuffer    67.859508

# Skin Temperature

In [15]:
skin_temperature = merged_df[['1-SKTA']]
skin_temperature = skin_temperature.groupby(["Participant","type","context"]).mean()
skin_temperature.rename(columns = {"1-SKTA":"skin_temperature"}, inplace = True)
skin_temperature.head(10)

skin_temperature
Participant type context                      
0001        2D   EndBuffer           12.963763
                 FrontBuffer         12.965864
                 HH                  12.963048
                 HL                  12.964674
                 LH                  12.966694
                 LL                  12.967358
            3D   EndBuffer           12.962631
                 FrontBuffer         12.955811
                 HH                  12.956955
                 HL                  12.963780

# EDA

### EDA Mean

In [16]:
#EDA MEAN
eda_features = merged_df[['EDA']]
eda_features = eda_features.groupby(["Participant","type","context"]).mean()
eda_features.rename(columns = {"EDA":"eda_mean"}, inplace = True)
display(eda_features.head(2))

eda_mean
Participant type context              
0001        2D   EndBuffer    5.155118
                 FrontBuffer  4.876494

### EDA Number of Peaks

In [17]:
def get_eda_features(eda):
    if len(eda > 15):
        try:
            features = nk.eda_process(eda = eda, sampling_rate = 250)
        except:
#             print("No peaks here")
            return 0
    else:
        return 0
    peaks = len(features['EDA']['SCR_Peaks_Indexes'])
    return peaks

In [18]:
#Number of peaks 
eda_peaks_df = pd.DataFrame([])
for participant in participants:
    data = merged_df.loc(axis = 0)[participant,:,:,:]
    d = data['EDA'].groupby(["Participant","type","context"]).apply(lambda x: get_eda_features(x))
    d = pd.DataFrame(d)
    eda_peaks_df = pd.concat([eda_peaks_df, d])


In [19]:
eda_peaks_df.rename(columns = {"EDA":"eda_no_of_peaks"}, inplace = True)
eda_features = eda_features.merge(eda_peaks_df, how = "left",right_index = True, left_index = True)

In [20]:
eda_features.head(2)

eda_mean  eda_no_of_peaks
Participant type context                               
0001        2D   EndBuffer    5.155118               19
                 FrontBuffer  4.876494               13

### EDA Power Spectrum

In [21]:
def compute_power_spectra(signal, band):
    try:
        x = np.fft.fft(signal - np.nanmean(signal))
        x = np.abs(x)
        freq = np.fft.fftfreq(len(signal), d = 0.004)
    except:
        return 0
    low, high = np.array(band)
    vals = [i for i in x if high>i>low]
    idx = [list(x).index(i) for i in vals]

    frequencies = freq[idx]
    power = np.sum((frequencies/len(signal))**2)
    return power

In [22]:
def compute_frequency_band_power(signal, band, sampling_rate):
    freq, power = ss.periodogram(signal - np.nanmean(signal), sampling_rate)
    low_f1, low_f2, high_f1, high_f2 = np.array(band)
    lfp_idx  = np.where((freq>=low_f1) & (freq>=low_f2))[0]
    hfp_idx  = np.where((freq>=high_f1) & (freq>=high_f2))[0] 
    lfp = np.trapz(power[lfp_idx], x = freq[lfp_idx])
    hfp = np.trapz(power[hfp_idx], x = freq[hfp_idx])
    
    return [lfp, hfp, lfp/hfp]

In [23]:
#High Frequecy and Low Frequency
bands =  [0.045, 0.15, 0.15,0.25]
eda_powers_df = pd.DataFrame([])
for participant in participants:
    data = merged_df.loc(axis = 0)[participant,:,:,:]
    eda_powers = data["EDA"].groupby(["Participant","type","context"]).\
                apply(lambda x:compute_frequency_band_power(x,bands,250) if len(x)>1 else pd.Series(np.nan))
    eda_powers = pd.DataFrame(eda_powers)
    eda_powers = pd.DataFrame(list(eda_powers["EDA"].values), index = eda_powers.index, 
                              columns = ["eda_lf","eda_hf","eda_lf_hf_ratio"])
    eda_powers_df = pd.concat([eda_powers_df, eda_powers])
   

In [24]:
eda_powers_df.head(2)

eda_lf    eda_hf  eda_lf_hf_ratio
Participant type context                                         
0001        2D   EndBuffer    0.016225  0.009354         1.734597
                 FrontBuffer  0.002777  0.001721         1.613714

In [25]:
eda_features = eda_features.merge(eda_powers_df, how = 'left', right_index = True, left_index = True)

In [26]:
eda_features.head(2)

eda_mean  eda_no_of_peaks    eda_lf    eda_hf  \
Participant type context                                                      
0001        2D   EndBuffer    5.155118               19  0.016225  0.009354   
                 FrontBuffer  4.876494               13  0.002777  0.001721   

                              eda_lf_hf_ratio  
Participant type context                       
0001        2D   EndBuffer           1.734597  
                 FrontBuffer         1.613714

### Removing the "Start" and "Stop" from the events

In [27]:
def change_events(df, log_events):
    df = df.reset_index("context")
    df = df[df['context']!="FrontBuffer"]
    df = df[df['context']!="EndBuffer"]
    df = df[df['context']!="Garbage"]
    df = df.set_index("context", append = True)
    return df

In [28]:
features_dataframe = {"ecg_hr":ecg_hr_features, "ecg_std":ecg_hrv_features, "skt":skin_temperature, "eda":eda_features}
all_features = pd.DataFrame([])
flag = False
for key, data in features_dataframe.items():
    print(key)
    f = change_events(data, log_events)
    features_dataframe[key] = f
    display(f.head(2))
    


ecg_hr


ecg_heart_rate
Participant type context                
0001        2D   HH             64.85144
                 HL             67.29351

ecg_std


ecg_hrv_std
Participant type context             
0001        2D   HH         78.254887
                 HL         95.744866

skt


skin_temperature
Participant type context                  
0001        2D   HH              12.963048
                 HL              12.964674

eda


eda_mean  eda_no_of_peaks    eda_lf    eda_hf  \
Participant type context                                                  
0001        2D   HH       2.845820               30  0.002634  0.001687   
                 HL       3.256573               18  0.001840  0.000855   

                          eda_lf_hf_ratio  
Participant type context                   
0001        2D   HH              1.561080  
                 HL              2.152029

Merging of all features in one dataframe

In [29]:
all_features = pd.DataFrame([])
flag= True
for key,value in features_dataframe.items():
    if flag:
        all_features = value
        flag = False
    else:
        all_features = all_features.merge(value, how = "left", right_index = True, left_index = True)
    display(all_features.head(2))    
# all_features.head(2)

ecg_heart_rate
Participant type context                
0001        2D   HH             64.85144
                 HL             67.29351

ecg_heart_rate  ecg_hrv_std
Participant type context                             
0001        2D   HH             64.85144    78.254887
                 HL             67.29351    95.744866

ecg_heart_rate  ecg_hrv_std  skin_temperature
Participant type context                                               
0001        2D   HH             64.85144    78.254887         12.963048
                 HL             67.29351    95.744866         12.964674

ecg_heart_rate  ecg_hrv_std  skin_temperature  \
Participant type context                                                  
0001        2D   HH             64.85144    78.254887         12.963048   
                 HL             67.29351    95.744866         12.964674   

                          eda_mean  eda_no_of_peaks    eda_lf    eda_hf  \
Participant type context                                                  
0001        2D   HH       2.845820               30  0.002634  0.001687   
                 HL       3.256573               18  0.001840  0.000855   

                          eda_lf_hf_ratio  
Participant type context                   
0001        2D   HH              1.561080  
                 HL              2.152029

In [30]:
all_features.head(2)

ecg_heart_rate  ecg_hrv_std  skin_temperature  \
Participant type context                                                  
0001        2D   HH             64.85144    78.254887         12.963048   
                 HL             67.29351    95.744866         12.964674   

                          eda_mean  eda_no_of_peaks    eda_lf    eda_hf  \
Participant type context                                                  
0001        2D   HH       2.845820               30  0.002634  0.001687   
                 HL       3.256573               18  0.001840  0.000855   

                          eda_lf_hf_ratio  
Participant type context                   
0001        2D   HH              1.561080  
                 HL              2.152029

Now, we want to take the datetimes from the original dataframe to match it up with the indexes of the feature

In [31]:
cols = merged_df.columns
time_df = merged_df.reset_index("Datetime")
time_df = time_df.drop(cols, axis = 1)
temp = time_df.reset_index("context")
temp = temp[(temp["context"]!="Garbage") & (temp["context"]!="FrontBuffer") & (temp["context"]!="EndBuffer")]
temp.set_index("context", append = True, inplace = True)
t = temp.reset_index()
t["Participant"] = t["Participant"].astype(int)
temp = t.set_index(["Participant","type","context"])
temp["Datetime"] = temp.groupby(["Participant","type","context"]).apply(lambda x: x["Datetime"][0])
temp = temp.drop_duplicates()
temp.head(2)

Datetime
Participant type context                    
1           2D   LL      2019-07-10 15:27:32
                 HH      2019-07-10 15:29:42

In [32]:
all_features.reset_index(inplace = True)
all_features["Participant"] = all_features["Participant"].astype(int)
all_features.set_index(["Participant","type","context"], inplace = True)
all_features.head(2)

ecg_heart_rate  ecg_hrv_std  skin_temperature  \
Participant type context                                                  
1           2D   HH             64.85144    78.254887         12.963048   
                 HL             67.29351    95.744866         12.964674   

                          eda_mean  eda_no_of_peaks    eda_lf    eda_hf  \
Participant type context                                                  
1           2D   HH       2.845820               30  0.002634  0.001687   
                 HL       3.256573               18  0.001840  0.000855   

                          eda_lf_hf_ratio  
Participant type context                   
1           2D   HH              1.561080  
                 HL              2.152029

In [33]:
final = all_features.join(temp)
final.set_index("Datetime", append = True, inplace = True)
final.head(3)

ecg_heart_rate  ecg_hrv_std  \
Participant type context Datetime                                           
1           2D   HH      2019-07-10 15:29:42       64.851440    78.254887   
                 HL      2019-07-10 15:34:02       67.293510    95.744866   
                 LH      2019-07-10 15:31:52       66.522944    94.265195   

                                              skin_temperature  eda_mean  \
Participant type context Datetime                                          
1           2D   HH      2019-07-10 15:29:42         12.963048  2.845820   
                 HL      2019-07-10 15:34:02         12.964674  3.256573   
                 LH      2019-07-10 15:31:52         12.966694  3.310807   

                                              eda_no_of_peaks    eda_lf  \
Participant type context Datetime                                         
1           2D   HH      2019-07-10 15:29:42               30  0.002634   
                 HL      2019-07-10 15:34:02               18  0.001840   
                 LH      2019-07-10 15:31:52               11  0.010295   

                                                eda_hf  eda_lf_hf_ratio  
Participant type context Datetime                                        
1           2D   HH      2019-07-10 15:29:42  0.001687         1.561080  
                 HL      2019-07-10 15:34:02  0.000855         2.152029  
                 LH      2019-07-10 15:31:52  0.005653         1.820969

Adding the column of context order and type index

In [34]:
final.sort_index(axis = 0, level = "Datetime", inplace = True)
final["context_order"] = final.groupby(["Participant","type"]).cumcount()+1
final.reset_index("type", inplace = True)
final["type_index"] = final["type"].apply(lambda x: 1 if x=="3D" else 0)
final.reset_index(inplace = True)
final.head(2)

,Participant,context,Datetime,type,ecg_heart_rate,ecg_hrv_std,skin_temperature,eda_mean,eda_no_of_peaks,eda_lf,eda_hf,eda_lf_hf_ratio,context_order,type_index
0,1,LL,2019-07-10 15:27:32,2D,63.845537,80.797150,12.967358,4.025697,19,0.005602,0.003178,1.763087,1,0
1,1,HH,2019-07-10 15:29:42,2D,64.851440,78.254887,12.963048,2.845820,30,0.002634,0.001687,1.561080,2,0


Adding the column of Difficulty and Perceptibility

In [35]:
final["Difficulty"] = final["context"].apply(lambda x: 1 if x[0]=="H" else 0)
final["Perceptibility"] = final["context"].apply(lambda x: 1 if x[1]=="H" else 0)

Dropping Participant 20

In [36]:
final = final[final["Participant"]!=20]

Saving the final dataframe as csv

In [37]:
# final.to_csv("/home/camar_temp/git-repos/buckets/bkt_prd_dsv_brain_therapy_raw/Features/Focused_Attention/features_FA_ANOVA_context_level.csv")